In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #For ploting graphs
from matplotlib.pyplot import figure

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/gufhtugu-publications-dataset-challenge/GP Orders - 2.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
df = pd.read_csv (r'../input/gufhtugu-publications-dataset-challenge/GP Orders - 2.csv',encoding='cp1252')  #encoding= 'unicode_escape')


df["Order Date"] = pd.DatetimeIndex(df["Order Date"])
df["Day_Name"] = df["Order Date"].dt.day_name()
df["Month_Name"] = df["Order Date"].dt.month_name()
df["Order Date"] = pd.DatetimeIndex(df["Order Date"])
df.set_index("Order Date", inplace=True)


print("Minimum date value", df.index.min(),"\n","Max date value", df.index.max())
df_2020 = df.loc["2020"]

#BREAKING DATA INTO DAY,TIME,MONTH
figure(num=None, figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(df_2020[df_2020["Order Status"]=="Completed"]["Month_Name"].value_counts(), marker="*", color="black")
plt.title("Top completed order months in 2020 year")
plt.grid()

#Data Showed MOST SALES ARE DURING SAT AND SUN COMPANY SHOULD SET MAX PROMOTION DURING THESE DAYS TO ATTRACT MORE CUSTOMERS

figure(num=None, figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(df_2020[df_2020["Order Status"]=="Completed"]["Day_Name"].value_counts(), marker="*", color="black")
plt.title("Top completed order days sells in 2020 year")
plt.grid()

from matplotlib.pyplot import figure
figure(num=None, figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')
df[df["Order Status"]=="Canceled"]["Month_Name"].value_counts().plot(kind="bar")
plt.grid()
plt.title("Order completion according to daytime")
#Text(0.5, 1.0, 'Order completion according to daytime')

from matplotlib.pyplot import figure
figure(num=None, figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(df_2020[df_2020["Order Status"]=="Canceled"]["Month_Name"].value_counts(), marker="*", color="black")
plt.title("Top canceled order months in 2020 year")
plt.grid()

#Sales according time Morning, Night, etc**

df["minute"] = df.index.minute
df["Hour"] = df.index.hour
df.loc[(df['Hour']<12),'Daytime']='Morning'
df.loc[(df['Hour']>=12)&(df['Hour']<17),'Daytime']='Afternoon'
df.loc[(df['Hour']>=17)&(df['Hour']<21),'Daytime']='Evening'
df.loc[(df['Hour']>=21)&(df['Hour']<23),'Daytime']='Night'
df["Daytime"].value_counts()


figure(num=None, figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')
df[df["Daytime"] =="Morning"]["Order Status"].value_counts().sort_values(ascending=False).head(10).plot(kind='bar')
plt.title("Order Status during morning time")
plt.grid()


figure(num=None, figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')
df[df["Daytime"] =="Night"]["Order Status"].value_counts().sort_values(ascending=False).head(10).plot(kind='bar')
plt.title("Order Status during morning time")
plt.grid()


figure(num=None, figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')
df[df["Daytime"] =="Evening"]["Order Status"].value_counts().sort_values(ascending=False).head(10).plot(kind='bar')
plt.title("Order Status during morning time")
plt.grid()

# completed orders during whole month**

figure(num=None, figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')
df[df["Order Status"]=="Completed"].resample("M").count()["Order Status"].plot(kind="bar")
plt.grid()


df[df["Order Status"]=="Completed"]["Daytime"].value_counts()
df[df["Order Status"]=="Completed"]["Month_Name"].value_counts()
df[df["Order Status"]=="Completed"]["Day_Name"].value_counts()


figure(num=None, figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')
df[df["Order Status"]=="Completed"]["Daytime"].value_counts().plot(kind="bar")
plt.grid()
plt.title("Order completion according to daytime")





In [ ]:
def tidy_split(df, column, sep='/', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.

    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df
#source https://stackoverflow.com/a/39946744   